In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for test data

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the target directory
        target_size=(150, 150),  # All images will be resized
        batch_size=32,
        class_mode='binary') # Use 'categorical' for multi-class classification

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

In [ ]:
import matplotlib.pyplot as plt

def plot_images(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

images = [train_generator[0][0][i] for i in range(5)]
plot_images(images)

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Use 'softmax' for multi-class classification
])

model.compile(loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multi-class
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # total train images / batch size
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50,  # total validation images / batch size
      verbose=2)

In [ ]:
eval_result = model.evaluate(validation_generator)
print(f"Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}")

In [ ]:
import numpy as np

# Assuming `test_images` is your dataset for predictions
predictions = model.predict(test_images)
predicted_class_indices = np.argmax(predictions, axis=-1)

# For binary classification
# predicted_class_indices = (predictions > 0.5).astype("int32")

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

def preprocess_image(image_path, target_size=(150, 150)):
    # Load the image
    img = load_img(image_path, target_size=target_size)
    # Convert the image to a numpy array
    img_array = img_to_array(img)
    # Scale the image pixels to be between 0 and 1
    img_array /= 255.0
    # Expand the dimensions so the input array shape becomes (1, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
test_image_paths = ["path/to/image1.jpg", "path/to/image2.jpg", ...]  # List of image paths
preprocessed_images = np.vstack([preprocess_image(path) for path in test_image_paths])

In [ ]:
# Assuming you have a list of test image paths
test_image_paths = ["path/to/image1.jpg", "path/to/image2.jpg", ...]

# Preprocess the images
preprocessed_images = np.vstack([preprocess_image(path) for path in test_image_paths])

# Make predictions
predictions = model.predict(preprocessed_images)

# Interpret predictions
# For binary classification:
predicted_classes_binary = (predictions > 0.5).astype(int)
# For multi-class classification:
predicted_classes_multi = np.argmax(predictions, axis=-1)

# Now you can use `predicted_classes_binary` or `predicted_classes_multi` depending on your task

In [ ]:
import matplotlib.pyplot as plt

def plot_images_with_predictions(images, true_labels, predictions, class_names=['Class 0', 'Class 1'], n=5):
    """
    Visualize the first n images with their predicted and true labels.
    
    :param images: The array of preprocessed images.
    :param true_labels: The true labels for the images.
    :param predictions: The model's predicted probabilities for class 1.
    :param class_names: The names of the classes.
    :param n: The number of images to display.
    """
    # Select the first n images, true labels, and predictions
    images = images[:n]
    true_labels = true_labels[:n]
    predictions = predictions[:n]
    
    # Set up the matplotlib figure and axes
    plt.figure(figsize=(2*n, 4))
    for i in range(n):
        # Display images
        plt.subplot(1, n, i+1)
        plt.imshow(images[i].squeeze(), cmap='gray')  # Adjust color map or remove it depending on your images
        plt.axis('off')
        
        # Get predicted label by checking if prediction > 0.5
        pred_label = 1 if predictions[i] > 0.5 else 0
        color = 'green' if pred_label == true_labels[i] else 'red'
        
        # Title will show predicted and true labels
        plt.title(f'Pred: {class_names[pred_label]}\nTrue: {class_names[true_labels[i]]}', color=color)

    plt.show()

# Assuming `preprocessed_images` is your array of test images and is not normalized between 0 and 1 for visualization purposes
# If your `preprocessed_images` were normalized, you might need to denormalize them for visualization.
plot_images_with_predictions(preprocessed_images, true_labels, predictions[:,0], class_names=['Not Class 1', 'Class 1'], n=5)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# true_labels: The true labels for your test dataset
# predicted_classes: The predicted labels for your test dataset (use predicted_classes_binary or predicted_classes_multi)

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_classes)
precision = precision_score(true_labels, predicted_classes, average='binary')  # For binary classification, for multi-class use 'micro', 'macro', or 'weighted'
recall = recall_score(true_labels, predicted_classes, average='binary')  # Same as precision
f1 = f1_score(true_labels, predicted_classes, average='binary')  # Same as precision

# Print metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
# Assuming `validation_generator` is your data generator

# Initialize an empty array to store the true labels
true_labels = []

# Iterate over the generator to extract true labels
for _, labels_batch in validation_generator:
    true_labels.extend(labels_batch)
    # Check if we've gone through all data
    if len(true_labels) >= validation_generator.samples:
        break

# Convert the list of true labels to a numpy array for further processing
true_labels = np.array(true_labels)

# If your labels are one-hot encoded, you might want to convert them to integer labels
# true_labels = np.argmax(true_labels, axis=1)  # Uncomment this line if needed

In [ ]:
# Predicting probabilities for the positive class
predictions = model.predict(images)

# Converting probabilities to class labels based on a threshold
predicted_classes = (predictions > 0.5).astype(int).flatten()

In [ ]:
# For multi-class classification
predicted_classes = np.argmax(predictions, axis=-1)


In [ ]:
# Assuming `validation_generator` is your data generator

# Initialize lists to store the images and their true labels
images = []
true_labels = []

# Iterate over the generator to extract images and true labels
for images_batch, labels_batch in validation_generator:
    images.extend(images_batch)
    true_labels.extend(labels_batch)
    # Check if we've gone through all data
    if len(true_labels) >= validation_generator.samples:
        break

# At this point, 'images' is a list of numpy arrays (the images), and 'true_labels' is a list of labels

# Optionally, convert lists to numpy arrays for further processing
images = np.array(images)
true_labels = np.array(true_labels)

In [ ]:
# Predicting probabilities for the positive class
predictions = model.predict(images)

# Converting probabilities to class labels based on a threshold
predicted_classes = (predictions > 0.5).astype(int).flatten()


In [ ]:
# For multi-class classification
predicted_classes = np.argmax(predictions, axis=-1)


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('dataset.csv')

In [ ]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def preprocess_image(image_path, target_size=(150, 150)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array /= 255.0  # Normalize the image
    return img_array

In [ ]:
# Assuming you have a function to convert labels to numeric format
def convert_labels(labels):
    # Implement label conversion here
    return numeric_labels

# Preprocess images
train_images = np.array([preprocess_image(path) for path in train_df['image_path']])
valid_images = np.array([preprocess_image(path) for path in valid_df['image_path']])

# Convert labels
train_labels = convert_labels(train_df['label'])
valid_labels = convert_labels(valid_df['label'])

In [ ]:
model.fit(train_images, train_labels, validation_data=(valid_images, valid_labels), epochs=10, batch_size=32)

In [ ]:
predictions = model.predict(valid_images)
# Convert predictions to labels based on your problem

# Evaluate the model
# Use accuracy_score, precision_score, recall_score, f1_score as shown previously

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
# Calculate metrics
accuracy = accuracy_score(valid_labels, predicted_labels)
precision = precision_score(valid_labels, predicted_labels, average='binary')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(valid_labels, predicted_labels, average='binary')  # Use 'macro' or 'weighted' for multi-class
f1 = f1_score(valid_labels, predicted_labels, average='binary')  # Use 'macro' or 'weighted' for multi-class

In [ ]:
cm = confusion_matrix(valid_labels, predicted_labels)
print(cm)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, ...)  # Add any other augmentations
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust based on your model's expected input size
    batch_size=32,
    class_mode='binary',  # or 'categorical' for multi-class classification
    color_mode='grayscale'  # Load images in grayscale
)

In [ ]:
img = load_img(image_path, target_size=(150, 150), color_mode='grayscale')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),  # Note the input shape is (150, 150, 1) for grayscale
    MaxPooling2D(2, 2),
    # Add more layers as needed
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # or Dense(num_classes, activation='softmax') for multi-class
])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),  # Note the input shape is (150, 150, 1) for grayscale
    MaxPooling2D(2, 2),
    # Add more layers as needed
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # or Dense(num_classes, activation='softmax') for multi-class
])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(grayscale_image.squeeze(), cmap='gray')
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# Add an output layer with softmax activation
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples // batch_size)

In [ ]:
# Unfreeze some layers in the base model
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model (necessary after making modifications to the layers)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Very low learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=fine_tune_epochs,  # Number of epochs to fine-tune for
          validation_data=validation_generator,
          validation_steps=validation_generator.samples // batch_size)

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('dataset.csv')

In [ ]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Adjust depending on your model's expected input
    return img_array

In [ ]:
X_train = np.array([preprocess_image(path) for path in train_df['image_path']])
X_valid = np.array([preprocess_image(path) for path in valid_df['image_path']])

# Assuming class labels are categorical and need to be one-hot encoded
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
y_train = encoder.fit_transform(train_df[['class']])
y_valid = encoder.transform(valid_df[['class']])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=32)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Change the last layer according to your problem
])

In [ ]:
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array /= 255.0  # Normalize images to [0, 1]
    return img_array

In [ ]:
import numpy as np

# Convert image paths to preprocessed image data
X = np.array([preprocess_image(path) for path in df['image_path']])

# Convert labels (assuming binary classification for simplicity)
y = df['class'].values

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # or 'categorical_crossentropy' for multi-class
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split for training and validation
)

# Compute quantities required for featurewise normalization
datagen.fit(X)

# Fit the model on the batches generated by datagen.flow()
model.fit(datagen.flow(X, y, batch_size=32, subset='training'),
          validation_data=datagen.flow(X, y, batch_size=32, subset='validation'),
          epochs=10)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array /= 255.0  # Normalize images to [0, 1]
    return img_array

In [ ]:
import numpy as np

X = np.array([preprocess_image(path) for path in df['image_path']])

# Assuming binary classification. Use appropriate encoding for multi-class scenarios.
y = df['class'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multi-class classification
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)